<a href="https://colab.research.google.com/github/TaizAlves/python/blob/master/aula15_Integrando_Google_Colab_e_Google_Planilhas_importadocota%C3%A7%C3%A3o_eDadosdasempresasfundamentalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ricos pelo Acaso
* Link para o vídeo: https://youtu.be/HlhbsrJ4_PQ

>Instalando `gspread`

In [4]:
!pip install gspread

>Importando as Bibliotecas

In [5]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials

>Realizando a Autenticação

In [6]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

>Criando uma planilha vazia

In [ ]:
gc.create('Planilha01')

<Spreadsheet 'Planilha01' id:19pRZ5GYD4cGGa-p2yoiuy05sKefT6w-XFUpi04jPlgE>

>Abrindo uma Planilha

In [ ]:
planilha = gc.open('Planilha01')

>Adicionando uma nova Página



In [ ]:
planilha.add_worksheet('nova_pagina', 10, 5)

<Worksheet 'nova_pagina' id:40256675>

>Selecionando uma Página

In [ ]:
pagina1 = planilha.get_worksheet(0)

>Apagando uma Página

In [ ]:
planilha.del_worksheet(pagina1)

{'replies': [{}],
 'spreadsheetId': '19pRZ5GYD4cGGa-p2yoiuy05sKefT6w-XFUpi04jPlgE'}

>Alterando os valores das células

>>1º passo: selecionar a página

In [ ]:
nova_pagina = planilha.get_worksheet(0)

>>`update_acell()`: pelo código da célula

In [ ]:
nova_pagina.update_acell('a1', 120)

{'spreadsheetId': '19pRZ5GYD4cGGa-p2yoiuy05sKefT6w-XFUpi04jPlgE',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': 'nova_pagina!A1',
 'updatedRows': 1}

>>`update_cell`: pelos números da linha e coluna

In [ ]:
nova_pagina.update_cell(2,2, 'qualquer texto')

{'spreadsheetId': '19pRZ5GYD4cGGa-p2yoiuy05sKefT6w-XFUpi04jPlgE',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': 'nova_pagina!B2',
 'updatedRows': 1}

>>`update_cells()`: por uma lista de valores

In [ ]:
lista_cel = nova_pagina.range('a1:b2')
lista_cel

[<Cell R1C1 '120'>,
 <Cell R1C2 ''>,
 <Cell R2C1 ''>,
 <Cell R2C2 'qualquer texto'>]

In [ ]:
valores = ['a','b','c','d']

for i, valor in enumerate(valores):
  lista_cel[i].value = valor

lista_cel

[<Cell R1C1 'a'>, <Cell R1C2 'b'>, <Cell R2C1 'c'>, <Cell R2C2 'd'>]

In [ ]:
nova_pagina.update_cells(lista_cel)

{'spreadsheetId': '19pRZ5GYD4cGGa-p2yoiuy05sKefT6w-XFUpi04jPlgE',
 'updatedCells': 4,
 'updatedColumns': 2,
 'updatedRange': 'nova_pagina!A1:B2',
 'updatedRows': 2}

###Carteira de Ações

In [8]:
plan_carteira = gc.open('carteira')

>Criando um DataFrame Pandas com uma planilha do Google Planilhas

In [9]:
import pandas as pd

In [10]:
carteira = pd.DataFrame(plan_carteira.sheet1.get_all_records())
carteira.set_index('Cod', inplace=True)

carteira

,Qte,Cotação,Entrada,Custo,No dia,Preço Atual,Lucro R$,Lucro %
Cod,,,,,,,,
VALE3,200,"44,66","54,33",10866,-4,8972,-1894,"-17,43"


###Salvando cotações do Yahoo Finance

>Adicionando uma página à planilha

In [11]:
plan_carteira.add_worksheet('Cotações Yahoo', 1500, 71)

<Worksheet 'Cotações Yahoo' id:498928782>

>Instalando o YFinance

In [20]:
!pip install yfinance

In [21]:
import yfinance as yf

In [22]:
tickers_ibov = "ABEV3.SA AZUL4.SA B3SA3.SA BBAS3.SA BBDC4.SA BBSE3.SA BPAC11.SA BRAP4.SA BRDT3.SA BRFS3.SA BRKM5.SA BRML3.SA BTOW3.SA CCRO3.SA CIEL3.SA CMIG4.SA COGN3.SA CRFB3.SA CSAN3.SA CSNA3.SA CVCB3.SA CYRE3.SA ECOR3.SA EGIE3.SA ELET6.SA EMBR3.SA ENBR3.SA EQTL3.SA FLRY3.SA GGBR4.SA GNDI3.SA GOAU4.SA GOLL4.SA HAPV3.SA HGTX3.SA HYPE3.SA IGTA3.SA IRBR3.SA ITSA4.SA ITUB4.SA JBSS3.SA KLBN11.SA LAME4.SA LREN3.SA MGLU3.SA MRFG3.SA MRVE3.SA MULT3.SA NTCO3.SA PCAR4.SA PETR4.SA QUAL3.SA RADL3.SA RAIL3.SA RENT3.SA SANB11.SA SBSP3.SA SMLS3.SA SULA11.SA SUZB3.SA TAEE11.SA TIMP3.SA TOTS3.SA UGPA3.SA USIM5.SA VALE3.SA VIVT4.SA VVAR3.SA WEGE3.SA YDUQ3.SA"

In [23]:
dados_yf = yf.download(tickers=tickers_ibov, period='5y')["Adj Close"]

[*********************100%***********************]  70 of 70 completed

1 Failed download:
- PCAR4.SA: No data found, symbol may be delisted


>Selecionando a página criada

In [24]:
pagina_yahoo = plan_carteira.get_worksheet(1)

>Instalando a biblioteca `gspread-dataframe`

In [25]:
!pip install gspread-dataframe

In [26]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe

>Preenchendo os dados na planilha

In [28]:
set_with_dataframe(pagina_yahoo, dados_yf, include_index=True)

###Dados Fundamentalistas

In [34]:
import requests

In [29]:
plan_carteira.add_worksheet('fundamentus', 500, 20)

<Worksheet 'fundamentus' id:1121439485>

In [30]:
pagina_fundamentus = plan_carteira.get_worksheet(2)

In [33]:
url = 'https://www.fundamentus.com.br/resultado.php'

In [35]:
header = {

  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",

  "X-Requested-With": "XMLHttpRequest"

}

r = requests.get(url, headers=header)

df = pd.read_html(r.text, decimal=',', thousands='.')[0]

In [36]:
set_with_dataframe(pagina_fundamentus, df)